In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dicc = {}
rm = 1
drm = 0.1
dicc["drm"] = 0.1

In [3]:
def error_propagation_tita_from_rm_L(L, dL):
    return math.sqrt((L / (drm**2 + L**2))**2 + (rm / (dL**2 + rm**2))**2)

In [4]:
def gen_dθr(L):
    return math.atan(rm/L) #+ error_propagation_tita_from_rm_L(L, 0.1)

In [5]:
def gen_dicc_with_deltas(dicc, data):
    #data is a pandas dataframe with columns t, x, y, r, tita
    dicc["dx"] = 0.1
    dicc["dy"] = 0.1
    dicc["dr"] = 0.1
    dicc["dt"] = 1/30
    L = data["r"].mean()
    dicc["dθr"] = gen_dθr(L)
    dicc["dθ0"] = dicc["dθr"]

In [6]:
print(dicc)
data = pd.read_csv("../datasets/plata l0/plata35.csv")
gen_dicc_with_deltas(dicc, data)
print(dicc)

{'drm': 0.1}
{'drm': 0.1, 'dx': 0.1, 'dy': 0.1, 'dr': 0.1, 'dt': 0.03333333333333333, 'dθr': 0.01961932845206364}


In [ ]:
f_data = format_pd(data=data, intento=1)

# Extract time and position data
tiempo = f_data["t"].values
posicion = distancia(f_data["x"].values)
errores_y = sigma_d(f_data["x"].values)
errores_x = np.full_like(tiempo, 20)  # Assuming a constant error in time

# Fit the quadratic model to the data
popt, pcov = curve_fit(modelo_cuadratico, tiempo, posicion, sigma=errores_y, absolute_sigma=True)

# Extract the fitted parameters and their uncertainties
a, b, c = popt
errores = np.sqrt(np.diag(pcov))

# Propagate errors
sigma_d_t = f_propagacion_errores(tiempo, popt, errores_x)

# Create a smoother time range for plotting the fitted curve
t_ajuste = np.linspace(min(tiempo), max(tiempo), 500)

# Use predefined labels for each dataset
label = dataset["label"]

# Plot the data points with error bars
plt.errorbar(tiempo, posicion, xerr=errores_x, yerr=errores_y, fmt='o', color=dataset["color"])

# Plot the quadratic fit
plt.plot(t_ajuste, modelo_cuadratico(t_ajuste, *popt), color=dataset["color"], linestyle='-')

# Plot the uncertainty bands
plt.fill_between(tiempo, 
                    modelo_cuadratico(tiempo, *popt) + sigma_d_t + errores_y, 
                    modelo_cuadratico(tiempo, *popt) - sigma_d_t - errores_y, 
                    color=dataset["color"], alpha=0.3)

# Label the dataset as a legend
plt.errorbar([], [], color=dataset["color"], label=label)

# Save the fitted parameters and their uncertainties
popts.append(popt)
erroress.append(errores)